# **Markov Chain Monte Carlo Simulation for Quantum Systems**

This notebook implements a Python-based Markov Chain Monte Carlo (MCMC) simulation to study quantum systems. Specifically, it focuses on approximating the ground-state wavefunction and correlation functions for two systems:

1. Quantum Harmonic Oscillator (QHO)

2. A canonically quantized version of the Friedmann-Lemaître-Robertson-Walker (FLRW) Universe
---

## **Methodology**

The simulation employs a **discretized path integral formulation** and integrates the following components:

### **Thermalization**
- The system begins in a random or predefined initial configuration.
- A thermalization phase ensures that the sampled configurations stabilize, removing any initial biases.
- Configurations from this phase are discarded to ensure unbiased sampling in later steps.

### **Sampling Configurations**
- Configurations are updated iteratively based on the Metropolis algorithm.
- A **Wick rotation** is applied to transform the action into a Euclidean formulation, enabling a probabilistic interpretation.
- Supports different action-modifying strategies, such as squaring or taking the absolute value of the action, to test alternative formulations.

### **Wavefunction Approximation**
- Position samples are accumulated into bins to construct a histogram.
- The normalized histogram approximates the probability distribution for the ground-state wavefunction.
- This approach reveals the expected Gaussian profile for the Quantum Harmonic Oscillator and potentially complex structures for the FLRW model.

### **Correlation Functions**
- The simulation calculates the two-point correlation function between positions at different time points.

### **Customizable Parameters**
- **Lattice Spacing (`a`)**: Controls the discretization of the time domain.
- **Number of Monte Carlo Steps (`NE`)**: Defines the total number of samples to draw.
- **Number of lattice points (`N`)**: Defines the total number of lattice time points. This means that `N*a` decides the total duration of the path.
- **Thermalization Steps (`therms`)**: Determines how many steps to discard for stabilization.
- **Physical Constants (`M`, `mu`, etc.)**: Include parameters like mass and angular frequency for the QHO or cosmological constants for the FLRW universe.


In [ ]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt

## Variables

N = 300  # Number of lattice points
a = 0.01  # Lattice spacing
NE = 100000  # Number of Monte Carlo samples
nbar = 10  # Number of Monte Carlo hits per lattice site
mu = np.sqrt(2)  # The constant in the first term of the harmonic potential
M = 0.5 #The mass factor in the kinetic energy term
delta = 0.2  # The range within which new values should be generated
k = 20  # Curvature
lamb = 1  # Cosmological constant
therms = 1000000  # Number of Thermalization Sweeps
therm_thresh = 0.1 #Amount that consecutive actions have to be within of eachother for the variable accum to be incremented
accum_thresh = 10 #Number of times accum needs to be incremented before the Thermalize() function aborts
init_path_lim = 15 #Range within which the initial path should be generated (is generated between -init_path_lim and +init_path_lim)

#Wavefunction-specific Parameters

range_lim = 4.5   #Positive negative limits of the range to be considered for binning the wavefunction
binsize = 0.025  # The size of the bins on either side when  approximating the wavefunction

#Correlation-specific Parameters

start_index = 1 #Lowest number to be considered during correlation  calculations
end_index = 51 #Highest number to be considered during correlation calculations
gap = 0.5 #Gap between sampled values for correlations
     
#Extra Parameters for transformations

beta = 10**(-3.47673)
#beta = 1
alpha = 1

special_sampling = "mod" #Type of transformation to apply to the action (std = wick rotation, sqr = wick rotation then square, mod = wick rotation then absolute value)
domain = "cosmo"            #String which defines whether the run is a cosmology or a qho run
simulation_type = "therm"   #String which defines what kind of simulation (wavefunction,correlation,energy) needs to be run
fix = 1                  #Whether to fix x[0] or not, takes binary input


xs = np.random.uniform(low=-init_path_lim, high=init_path_lim, size=N)  ## The original path that we start from (random numbers)

if fix:
    xs[0] = 0  # Fixes the first position point to 0 as required by the paper

#For saving the files
def get_next_available_filename(base_filename):
    """Check for existing files and return the next available filename."""
    n = 1
    filename = f"{base_filename}{n}.npy"

    while os.path.exists(filename):
        n += 1
        filename = f"{base_filename}{n}.npy"

    return filename

## Action for the entire path (generalized for special sampling cases)

def calculate_total_action(inp):

    global action_sign

    if domain == "cosmo":

      term1 = np.sum((np.square(np.diff(inp)))) / (2 * a)
      term2 = (lamb * np.sum((np.square(inp))) * a) / 2
      term3 = k * np.sum(np.cbrt(np.square(inp))) * a
      action = 0


      #Returns just the action for the 'std' case (the mod case is dealt with in the MonteCarloHit() function)

      if (special_sampling == "std" or special_sampling == "mod"):
        action = term1 - term2 + term3
        action_sign = np.sign(action)

        return action

      #the action squared for the 'sqr' case,

      elif (special_sampling == "sqr"):
        action = term1 - term2 + term3
        action_sign = np.sign(action)

        return action**2

    elif domain == "qho":

      term1 = M*np.sum((np.square(np.diff(xs))))/(2*a)
      term2 = ((mu**2*np.sum((np.square(xs))))/2)*a
      action = 0


      #Returns just the action for the 'std' case (the mod case is dealt with in the MonteCarloHit() function)

      if (special_sampling == "std" or special_sampling == "mod"):
        action = term1 + term2
        action_sign = np.sign(action)

        return action

      #the action squared for the 'sqr' case,

      elif (special_sampling == "sqr"):
        action = term1 + term2
        action_sign = np.sign(action)

        return action**2


## Differential Action: Only for the affected index
def Action_diff(inp, index):

    if domain == "cosmo":
      term1 = 0.0
      term2 = (lamb * (inp[index]**2) * a) / 2
      term3 = k * np.cbrt(inp[index]**2) * a

      if index > 0:
        term1 += (inp[index] - inp[index-1])**2 / (2 * a)
      if index < N - 1:
        term1 += (inp[index+1] - inp[index])**2 / (2 * a)

      return term1 - term2 + term3

    elif domain == "qho":
      term1 = 0
      term2 = (mu**2)*(inp[index]**2)*a/2

      if index > 0:
        term1 += M*(inp[index] - inp[index-1])**2 / (2 * a)
      if index < N - 1:
        term1 += M*(inp[index+1] - inp[index])**2 / (2 * a)

      return term1 + term2

## Monte Carlo hit: Recalculate only the affected parts
def MonteCarloHit(x, index, xs, hits, newx, total_action):

    global action_sign

    currentx = x
    currentxs = xs
    current_action = total_action
    actual_action = 0

    # Calculate original action for the current index only
    old_action_contrib = Action_diff(currentxs, index)

    # Temporarily update the path and calculate the new action for this index
    currentxs[index] = newx
    new_action_contrib = Action_diff(currentxs, index)

    # Revert the path back to its original state
    currentxs[index] = currentx

    # Calculate the new total action by simply adding the new contribution and subtracting the old one in the standard case,

    if (special_sampling == "std"):
        new_total_action = current_action - old_action_contrib + new_action_contrib

        # Accept or reject the new position based on the Metropolis criterion

        if current_action > new_total_action:
            currentx = newx
            current_action = new_total_action
        elif random.uniform(0, 1) <= np.exp(-(new_total_action - current_action)):
            currentx = newx
            current_action = new_total_action

        return currentx, current_action

    #Expanding the formula (S-old+new)**2 in the squared case:

    elif (special_sampling == "sqr"):
        new_total_action = (current_action) + (old_action_contrib)**2 + (new_action_contrib)**2 - 2*old_action_contrib*np.sqrt(current_action) + 2*new_action_contrib*np.sqrt(current_action) - 2*new_action_contrib*old_action_contrib

        # Accept or reject the new position based on the Metropolis criterion (taking into account beta)

        if beta*current_action > beta*new_total_action:
            currentx = newx
            current_action = new_total_action
        elif random.uniform(0, 1) <= np.exp(-beta*(new_total_action - current_action)):
            currentx = newx
            current_action = new_total_action

        return currentx, current_action


    #Calculating the mod of the resulting action

    elif (special_sampling == "mod"):

        #Takes the global variable action_sign to restore the value of the current_action before taking the mod
        #,then accounts for the contributions from Action_diff

        actual_action = action_sign*current_action - old_action_contrib + new_action_contrib

        #Updates the global variable action_sign according to the new actual action (pre-modding)

        action_sign = np.sign(actual_action)

        #Updates the current and new action variables before comparison to restore modding
        new_total_action = np.abs(actual_action)
        current_action = np.abs(current_action)

        # Accept or reject the new position based on the Metropolis criterion (taking into account beta)

        if alpha*current_action > alpha*new_total_action:
            currentx = newx
            current_action = new_total_action
        elif random.uniform(0, 1) <= np.exp(-alpha*(new_total_action - current_action)):
            currentx = newx
            current_action = new_total_action

        return currentx, current_action

## Thermalize function
def Thermalize(init):
    currentxs = init
    newxs = np.zeros_like(currentxs)
    current = init[0]
    total_action = calculate_total_action(currentxs)  # Initial action calculation
    action_before = 0
    action_after = 0
    accum = 0

    for i in range(therms):
        newxs = currentxs + np.random.uniform(-delta, delta, size=currentxs.shape)

        for j in range(fix, N):
            current, total_action = MonteCarloHit(currentxs[j], j, currentxs, nbar, newxs[j], total_action)
            currentxs[j] = current

        if not (i % 100):
            action_after = total_action
            print(f"{(i / therms) * 100:.2f}% THERMALIZED")
            print(total_action)
            #plt.plot(currentxs, np.arange(0, currentxs.size))

            #Checks whether consecutive printed actions are within 'therm_thresh' of eachother and adds 1 to accum if they are
            if i != 0 and np.abs(np.abs(action_before) - np.abs(action_after) <= therm_thresh):
              accum += 1
              print(f'Actions have been within {therm_thresh} of eachother {accum} times')

            #Exits the function if accum exceeds accum_thresh
            if accum > accum_thresh:
              return currentxs

            action_before = total_action

            plt.show()

    return currentxs

## Simulation Run function
def SimulateWavefunction(init):
    # Define the bin edges based on the range limits and bin size
    pos = np.arange(-range_lim, range_lim + binsize * 2, binsize * 2)
    wavefunc = np.zeros(len(pos) - 1)  # Adjust wavefunc to match the number of bins
    currentxs = init
    newxs = np.zeros_like(currentxs)
    total_action = calculate_total_action(currentxs)  # Initial total action

    for i in range(NE*nbar):
        # Generate new positions
        newxs = currentxs + np.random.uniform(-delta, delta, size=currentxs.shape)

        for j in range(fix, N):
            current, total_action = MonteCarloHit(currentxs[j], j, currentxs, nbar, newxs[j], total_action)
            currentxs[j] = current

        if not (i % nbar):

            # Binning the wavefunction using np.histogram (every nbar steps of i)
            counts, _ = np.histogram(currentxs[fix:], bins=pos)  # Get counts in each bin based on edges in pos
            wavefunc += counts  # Accumulate counts

        if not (i % 10):
            print(f"{(i / (NE*nbar)) * 100:.2f}% DONE")
            print(total_action)
            #plt.plot(pos[:-1], wavefunc / np.sum(wavefunc * binsize * 2))  # Use pos[:-1] for plotting
            plt.show()

    return wavefunc/np.trapz(wavefunc), currentxs

## Simulation Run function
def SimulateCorrelations(init):

    correlarray = np.zeros(end_index - start_index)
    currentxs = init
    newxs = np.zeros_like(currentxs)
    total_action = calculate_total_action(currentxs)  # Initial total action
    for i in range(NE*nbar):
        # Generate new positions
        newxs = currentxs + np.random.uniform(-delta, delta, size=currentxs.shape)
        
        if not (i % nbar):
            correlarray += currentxs[start_index] * currentxs[start_index:start_index + (end_index - start_index)]

        for j in range(fix,N):
            current, total_action = MonteCarloHit(currentxs[j], j, currentxs, nbar, newxs[j], total_action)
            currentxs[j] = current

        if not (i % 100):
            print(str((i/(NE*nbar))*100) + " PERCENT DONE")
            #plt.plot(np.arange(start_index*gap , end_index*gap , gap), correlarray)
            plt.show()

    return correlarray/(NE), currentxs


if simulation_type == "wf":
    
    #thermalized = Thermalize(xs)
    wave, xs = SimulateWavefunction(therm_std_1)
    plt.plot(np.arange(-range_lim + binsize, range_lim, binsize * 2), wave)

    # Base filename for the wave file
    base_filename = f'{domain}_wave_S_{special_sampling}_Lamb_{lamb}_curv_{k}_NE_{NE}_N_{N}_a_{a}_bin_{binsize}_therms_{therms}_mu_{mu}_beta_{beta}_pos_neg_range_{range_lim}__delta_{delta}_nbar_{nbar}_'
    filename = get_next_available_filename(base_filename)

    # Save the wave data to the next available file
    np.save(filename, wave)

elif simulation_type == "cr":
    
    #thermalized = Thermalize(xs)
    correlations, xs = SimulateCorrelations(therm_std_3)
    plt.plot(np.arange(start_index*gap , end_index*gap , gap), correlations)

    # Base filename for the wave file
    base_filename = f'{domain}_cor_S_{special_sampling}_N_{N}_NE_{NE}_a_{a}_bin_{binsize}_therms_{therms}_from_{start_index*gap}_to_{end_index*gap}_interval_{gap}_delta_{delta}_nbar_{nbar}_'
    filename = get_next_available_filename(base_filename)

    np.save(filename, correlations)
    
elif simulation_type == "therm":
    for i in range(5):
        print(f'ITERATION {i+1} WITH N = {N}')
        accum_thresh = therms
        thermalized = Thermalize(xs)
    
        # Base filename for the wave file
        base_filename = f'{domain}_thermalized_array_therms_{therms}_S_{special_sampling}_N_{N}_NE_{NE}_a_{a}_delta_{delta}_nbar_{nbar}_'
        filename = get_next_available_filename(base_filename)

        np.save(filename, thermalized)